<a href="https://colab.research.google.com/github/RyuMinHo/GAI_project/blob/main/image_gradio_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vllm transformers triton PyPDF2 Pillow sentence_transformers numpy typing faiss-gpu semchunk gradio docling pymupdf4llm fitz frontend tools
!python -m spacy download en_core_web_sm

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 101.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
import gradio as gr
from PyPDF2 import PdfReader
import semchunk
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from vllm import LLM, SamplingParams
from PIL import Image
import pymupdf4llm
import pymupdf as fitz
import os
from typing import List, Tuple

def init_models():
    try:
        # GPU 메모리 관리 설정 추가
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"CUDA device available: {torch.cuda.get_device_name(0)}")

        print("Loading LLaVA model...")
        llm = LLM(model="llava-hf/llava-v1.6-mistral-7b-hf",
                  dtype='half',
                  max_model_len=8192,
                  device="cuda" if torch.cuda.is_available() else "cpu",
                  gpu_memory_utilization=0.9)  # GPU 메모리 사용량 제한

        print("Loading SentenceTransformer...")
        embedder = SentenceTransformer('all-MiniLM-L6-v2',
                                     device="cuda" if torch.cuda.is_available() else "cpu")

        print("All models initialized successfully!")
        return llm, embedder
    except Exception as e:
        print(f"Error during model initialization: {str(e)}")
        return None, None

# Initialize models as global variables
print("Starting model initialization...")
GLOBAL_LLM, GLOBAL_EMBEDDER = init_models()
if GLOBAL_LLM is None or GLOBAL_EMBEDDER is None:
    raise RuntimeError("Failed to initialize one or more required models")
print("Available models:", "LLaVA" if GLOBAL_LLM else "None")

Starting model initialization...
CUDA device available: NVIDIA A100-SXM4-40GB
Loading LLaVA model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

WARNING 12-01 10:29:17 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 12-01 10:29:27 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-01 10:29:27 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llava-hf/llava-v1.6-mistral-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-v1.6-mistral-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=F

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

All models initialized successfully!
Available models: LLaVA


In [ ]:
import gradio as gr
import base64
import os  # 파일 경로 관리
from PIL import Image  # 이미지 처리 라이브러리
import io  # 파일 입출력 핸들링
import torch
import faiss
import numpy as np
from vllm.sampling_params import SamplingParams
from PyPDF2 import PdfReader

# 세션별 FAISS 인덱스 및 청크를 관리하는 클래스
class SessionIndexManager:
    def __init__(self, embedder):
        """
        초기화 메서드: 세션별 인덱스와 청크를 저장할 딕셔너리를 생성
        """
        self.embedder = embedder  # 텍스트 임베딩 생성기
        self.session_indices = {}  # 세션별 FAISS 인덱스 저장
        self.session_chunks = {}   # 세션별 문서 청크 저장

    def create_session_index(self, session_id):
        """
        새로운 세션 ID를 위한 FAISS 인덱스를 생성
        """
        dimension = self.embedder.get_sentence_embedding_dimension()  # 임베딩 차원 가져오기
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반의 평평한 FAISS 인덱스 생성
        self.session_indices[session_id] = index  # 생성된 인덱스를 세션 딕셔너리에 저장
        self.session_chunks[session_id] = []  # 해당 세션의 청크 리스트 초기화

    def add_document(self, session_id, chunks):
        """
        세션에 문서 청크를 추가하고 FAISS 인덱스에 추가
        """
        if session_id not in self.session_indices:  # 세션 인덱스가 없으면 생성
            self.create_session_index(session_id)

        embeddings = self.embedder.encode(chunks)  # 청크를 임베딩 벡터로 변환
        self.session_indices[session_id].add(np.array(embeddings))  # FAISS 인덱스에 벡터 추가
        self.session_chunks[session_id].extend(chunks)  # 청크 리스트에도 저장

    def search(self, session_id, query, k=5):
        """
        세션 인덱스를 사용해 쿼리와 유사한 상위 k개의 청크 검색
        """
        if session_id not in self.session_indices:  # 세션이 존재하지 않으면 빈 리스트 반환
            return []

        query_embedding = self.embedder.encode([query])  # 쿼리를 임베딩 벡터로 변환
        D, I = self.session_indices[session_id].search(
            np.array(query_embedding), k  # 상위 k개의 유사 벡터 검색
        )
        # 검색된 인덱스를 기반으로 세션 청크를 반환
        return [self.session_chunks[session_id][i] for i in I[0]]

    def delete_session(self, session_id):
        """
        특정 세션의 FAISS 인덱스와 청크 삭제
        """
        if session_id in self.session_indices:
            del self.session_indices[session_id]  # 세션 인덱스 제거
        if session_id in self.session_chunks:
            del self.session_chunks[session_id]  # 세션 청크 제거

# 전역 인덱스 매니저 생성
index_manager = SessionIndexManager(GLOBAL_EMBEDDER)

# 전역 state 선언
state = {
    "sessions": {
        "Example": {
            "history": [],
            "mode": "General Chat",
            "mode_locked": False  # 초기값은 False
        }
    },
    "current_session": "Example"
}


def process_pdf_to_chunks(file):
    """
    PDF 파일을 읽어서 텍스트를 추출하고 청크로 분할
    """
    try:
        # PDF 파일 읽기
        pdf_reader = PdfReader(file)

        # 모든 페이지의 텍스트 추출
        text_chunks = []
        for page in pdf_reader.pages:
            text = page.extract_text()
            # 기본적인 텍스트 전처리
            text = text.strip()
            if text:  # 빈 페이지가 아닌 경우만 추가
                text_chunks.append(text)

        print(f"Successfully processed PDF with {len(text_chunks)} pages")
        return text_chunks

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return []

def process_message(message, file, mode, history, session_name):
    def post_process_response(response):
        sentences = response.split('.')
        unique_sentences = []
        for s in sentences:
            s = s.strip()
            if s and s not in unique_sentences:
                unique_sentences.append(s)
        return '. '.join(unique_sentences)

    if mode == "General Chat":
        if not file:  # Text only
            try:
                if history and history[-1][1]:  # If there's previous context
                    prompt = f"""Previous context: {history[-1][1]}

New question: {message}

Please answer the new question considering the previous context."""
                else:
                    prompt = message

                sampling_params = SamplingParams(
                    temperature=0.15,
                    max_tokens=512,
                    top_p=0.85,
                    repetition_penalty=1.18
                )

                response = GLOBAL_LLM.generate(
                    prompts=[prompt],
                    sampling_params=sampling_params
                )
                return post_process_response(response[0].outputs[0].text.strip())
            except Exception as e:
                return f"Error processing text: {str(e)}"

        else:  # Image processing
            try:
                with Image.open(file) as image:
                    if image.mode != 'RGB':
                        image = image.convert('RGB')

                    question = message if message.strip() else "Please describe this image."

                    prompt = f"""[INST] <image>
                    Please analyze and describe this image in detail:

                    Question: {question}

                    Note: Please provide objective and detailed observations based only on what is clearly visible. [/INST]"""

                    inputs = {
                        "prompt": prompt,
                        "multi_modal_data": {"image": image}
                    }

                    sampling_params = SamplingParams(
                        temperature=0.15,
                        max_tokens=512,
                        top_p=0.85,
                        repetition_penalty=1.18
                    )

                    try:
                        outputs = GLOBAL_LLM.generate(inputs, sampling_params=sampling_params)
                        if outputs and len(outputs) > 0:
                            response = outputs[0].outputs[0].text.strip()
                            if not response:
                                return "Failed to generate response."
                            return post_process_response(response)
                    finally:
                        del inputs
                        if 'outputs' in locals():
                            del outputs
                        import gc
                        gc.collect()
                        torch.cuda.empty_cache()

            except Exception as e:
                print(f"Debug - Image processing error: {str(e)}")
                return f"Error processing image: {str(e)}"

    else:  # RAG Chat mode
        try:
            # PDF 파일이 업로드된 경우
            if file and file.name.endswith('.pdf'):
                chunks = process_pdf_to_chunks(file)
                if not chunks:
                    return "Failed to process PDF file. Please check if the file is valid."
                index_manager.add_document(session_name, chunks)
                return f"PDF processed successfully. Extracted {len(chunks)} pages. You can now ask questions about the document."

            # PDF가 업로드되었는지 확인
            if session_name not in index_manager.session_indices:
                return "Please upload a PDF document first."

            # 질문에 대한 관련 청크 검색
            relevant_chunks = index_manager.search(session_name, message)
            if not relevant_chunks:
                return "No relevant information found in the document."

            # 검색된 청크들을 기반으로 프롬프트 생성
            context = "\n".join(relevant_chunks)
            prompt = f"""Please answer the question based on the following context:

Context:
{context}

Question: {message}

Note:
- Use only the information from the given context
- Don't mention uncertain information
- If information is not in the context, acknowledge that you don't know"""

            sampling_params = SamplingParams(
                temperature=0.15,
                max_tokens=512,
                top_p=0.85,
                repetition_penalty=1.18
            )

            response = GLOBAL_LLM.generate(
                prompts=[prompt],
                sampling_params=sampling_params
            )
            return post_process_response(response[0].outputs[0].text.strip())

        except Exception as e:
            return f"Error in RAG processing: {str(e)}"

def send_message(message, file, session_name, mode, history):
    global state
    if not message.strip() and not file:
        return history, "", None, "", ""

    try:
        if session_name in state["sessions"] and not state["sessions"][session_name]["mode_locked"]:
            if message.strip() or file:
                state["sessions"][session_name]["mode_locked"] = True
                state["sessions"][session_name]["mode"] = mode

        if file:
            response = process_message(message, file, mode, history, session_name)
            if message.strip():
                # 메시지가 있는 경우 파일과 메시지 함께 표시
                history.append(((file.name, file), message.strip()))
            else:
                # 파일만 있는 경우 파일만 표시 (빈 메시지 없이)
                history.append(((file.name, file), None))
            history.append((None, response))
        else:
            if message.strip():
                response = process_message(message, file, mode, history, session_name)
                history.append((None, message))
                history.append((None, response))

        if session_name in state["sessions"]:
            state["sessions"][session_name]["history"] = history

        return history, "", None, "", ""

    except Exception as e:
        print(f"Error sending message: {str(e)}")
        return history, "", None, "", ""

def create_ui():
    custom_css = """
    .message-box {
        display: flex;
        align-items: center;
        gap: 0.5rem;
    }
    .file-btn {
        max-width: 40px;
    }
    .send-btn {
        max-width: 40px;
    }
    .selected-file {
        margin: 0.5rem 0;
        padding: 0.3rem;
        background: #f0f0f0;
        border-radius: 4px;
        font-size: 0.9em;
    }
    """

    with gr.Blocks(css=custom_css) as demo:
        with gr.Row():
            with gr.Column(scale=1):
                new_session_btn = gr.Button("+ New Session")
                session_title_input = gr.Textbox(
                    label="Session Title",
                    visible=False
                )
                with gr.Column(elem_classes="session-container"):
                    gr.Markdown("Sessions")
                    session_list = gr.Radio(
                        choices=["Example"],
                        value="Example",
                        interactive=True,
                        label=""
                    )
                    delete_btn = gr.Button("🗑️ Delete Session")

            with gr.Column(scale=3):
                current_title = gr.Markdown("## Example")

                with gr.Row():
                    chat_mode = gr.Radio(
                        choices=["General Chat", "RAG Chat"],
                        value="General Chat",
                        interactive=True,
                        label=""
                    )

                with gr.Column():
                    chatbot = gr.Chatbot(
                        height=400,
                        render_markdown=True,
                        show_copy_button=True,
                        bubble_full_width=False
                    )

                    with gr.Row():
                        file_upload = gr.UploadButton(
                            "📎",
                            file_types=[".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG", ".pdf", ".PDF"],
                            scale=1
                        )
                        msg = gr.Textbox(
                            show_label=False,
                            placeholder="Type your message...",
                            container=False,
                            scale=44
                        )
                        send_btn = gr.Button(
                            "↑",
                            scale=1
                        )

                    with gr.Row():
                        clear_btn = gr.Button("Clear Chat")

                    # Selected File을 두 개의 컴포넌트로 분리
                    with gr.Row(visible=True) as general_file_info:
                        selected_image = gr.Textbox(
                            label="Selected Image",
                            interactive=False
                        )

                    with gr.Row(visible=False) as rag_file_info:
                        selected_pdf = gr.Textbox(
                            label="Selected PDF",
                            interactive=False
                        )

        def show_title_input():
            return gr.update(visible=True)

        def add_session(title):
            global state
            if not title or title in state["sessions"]:
                return (
                    gr.update(visible=False),
                    gr.update(choices=list(state["sessions"].keys()),
                            value=state["current_session"])
                )

            new_sessions = {title: {
                "history": [],
                "mode": "General Chat",
                "mode_locked": False  # 새 세션에서 mode_locked 초기화
            }}
            new_sessions.update(state["sessions"])
            state["sessions"] = new_sessions
            state["current_session"] = title

            choices = list(state["sessions"].keys())

            return (
                gr.update(visible=False),
                gr.update(choices=choices, value=title)
            )

        def switch_session(session_name):
            global state
            if session_name in state["sessions"]:
                state["current_session"] = session_name
                return (
                    f"## {session_name}",
                    state["sessions"][session_name]["history"],
                    state["sessions"][session_name]["mode"]
                )
            return current_title, [], chat_mode.value

        def delete_session(session_name):
            global state
            if len(state["sessions"]) <= 1:
                gr.Warning("Cannot delete the last session")
                return (
                    gr.update(choices=list(state["sessions"].keys()),
                            value=session_name),
                    current_title,
                    chatbot,
                    chat_mode
                )

            if session_name in state["sessions"]:
                del state["sessions"][session_name]
                index_manager.delete_session(session_name)
                next_session = next(iter(state["sessions"].keys()))
                state["current_session"] = next_session
                return (
                    gr.update(choices=list(state["sessions"].keys()),
                            value=next_session),
                    f"## {next_session}",
                    state["sessions"][next_session]["history"],
                    state["sessions"][next_session]["mode"]
                )
            return (
                gr.update(choices=list(state["sessions"].keys())),
                current_title,
                chatbot,
                chat_mode
            )

        def clear_chat():
            global state
            session_name = state["current_session"]
            if session_name in state["sessions"]:
                state["sessions"][session_name]["history"] = []
                index_manager.delete_session(session_name)
                # mode_locked는 유지 (초기화하지 않음)
            return [], "", ""  # chatbot, selected_image, selected_pdf 초기화

        def chat_mode_change(mode, session_name):
            global state
            if session_name in state["sessions"]:
                # 채팅 기록이 있거나 모드가 잠겨있으면 모드 변경 불가
                if state["sessions"][session_name]["mode_locked"]:
                    gr.Warning("Cannot change mode after chat has started. Please create a new session.")
                    # 현재 모드를 유지하도록 업데이트
                    current_mode = state["sessions"][session_name]["mode"]
                    is_general = current_mode == "General Chat"
                    file_types = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"] if is_general else [".pdf", ".PDF"]

                    return [
                        gr.update(value=current_mode),  # 라디오 버튼 값을 현재 모드로 되돌림
                        gr.update(file_types=file_types),
                        gr.update(visible=is_general),
                        gr.update(visible=not is_general)
                    ]

                state["sessions"][session_name]["mode"] = mode
                is_general = mode == "General Chat"
                file_types = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"] if is_general else [".pdf", ".PDF"]

                return [
                    gr.update(value=mode),  # 새로운 모드로 업데이트
                    gr.update(file_types=file_types),
                    gr.update(visible=is_general),
                    gr.update(visible=not is_general)
                ]

        def update_selected_file(file):
            """파일 종류에 따라 적절한 텍스트박스에 파일명 표시"""
            if file.name.lower().endswith(('.jpg', '.jpeg', '.png')):
                return file.name, "", True, False  # image_name, pdf_name, image_visible, pdf_visible
            else:
                return "", file.name, False, True



        # Event listeners
        new_session_btn.click(
            show_title_input,
            outputs=session_title_input
        )

        session_title_input.submit(
            add_session,
            inputs=[session_title_input],
            outputs=[session_title_input, session_list]
        )

        session_list.change(
            switch_session,
            inputs=[session_list],
            outputs=[current_title, chatbot, chat_mode]
        )

        delete_btn.click(
            delete_session,
            inputs=[session_list],
            outputs=[session_list, current_title, chatbot, chat_mode]
        )

        chat_mode.change(
            chat_mode_change,
            inputs=[chat_mode, session_list],  # session_list 추가
            outputs=[
                chat_mode,  # 라디오 버튼 자체를 출력에 추가
                file_upload,
                general_file_info,
                rag_file_info
            ]
        )

        file_upload.upload(
            fn=update_selected_file,
            inputs=[file_upload],
            outputs=[selected_image, selected_pdf]
        )

        send_btn.click(
            send_message,
            inputs=[msg, file_upload, session_list, chat_mode, chatbot],
            outputs=[
                chatbot,  # 채팅 히스토리
                msg,      # 메시지 입력창 초기화
                file_upload,  # 파일 업로드 초기화
                selected_image,  # 이미지 파일명 초기화
                selected_pdf    # PDF 파일명 초기화
            ]
        )

        msg.submit(
            send_message,
            inputs=[msg, file_upload, session_list, chat_mode, chatbot],
            outputs=[
                chatbot,
                msg,
                file_upload,
                selected_image,
                selected_pdf
            ]
        )

        clear_btn.click(
            clear_chat,
            outputs=[chatbot, selected_image, selected_pdf]
        )

        return demo

if __name__ == "__main__":
    demo = create_ui()
    demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://133d7a713216c62c8b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
